# Splitsville: Fun with the New **regex** Module

## Rex Dwyer
## TriPython Monthly Meeting
## April 28, 2016

The Python regular expression module **re** has a number of deficiencies.  There are several cases where its behavior differs from Perl, the gold standard for regular expressions. The **regex** module is intended to replace **re**. **regex** can be found at https://pypi.python.org/pypi/regex/2016.04.25 In addition to fixing some bugs, it adds a lot of new features. This notebook reviews a few 'advanced' features of regular expressions, then introduces some of the new features of **regex**.  It's not intended to be comprehensive.

First, let's review a few things from the old **re** module that haven't changed.

In [22]:
import re

t = 'The quick brown fox born on 1/23/2013 jumped over the lazy dog born on 10/6/10.'
print t

match(r'The',t)

The quick brown fox born on 1/23/2013 jumped over the lazy dog born on 10/6/10.


### match and search
Use **r'...'** (raw strings) for patterns, because we like to use elements like **\w**, and we don't want to have to type **\\\\w**.
**match** looks for the pattern at the very beginning of the target string.


(Normally if we were going to reuse a pattern, we'd compile it and use the **match** method of the resulting pattern like this:

In [23]:
p = compile(r'The')
p.match(t)

But in this tutorial, we will not compile.)

This one doesn't match, because **the** is not at the beginning of the target string.  It returns **None**, which of course doesn't print any output.

In [26]:

match(r'he',t)


**search** looks at the whole string.  It finds the **the** before **lazy dog**.

In [27]:

search(r'the',t)


**search** and **match** can be made case-insensitive.

In [28]:

search(r'the',t, IGNORECASE)


In [29]:

search(r'(?i)the',t)


### **findall** and **finditer**
**findall** lists every match to the pattern, but doesn't give the position.  **I** is short for **IGNORECASE**.

In [30]:

findall(r'the',t, I)


['The', 'the']

**finditer** returns an iterator over all matches of the pattern in the string.

In [31]:

finditer(r'the',t, I)


<callable-iterator at 0x7235c18>

In [32]:

list(finditer(r'the',t, I))


[<_sre.SRE_Match at 0x73763d8>, <_sre.SRE_Match at 0x7376370>]

In [33]:

list(finditer(r'(brown|lazy) (dog|fox)',t))


[<_sre.SRE_Match at 0x71b2e88>, <_sre.SRE_Match at 0x71b2f10>]

The **X** option lets us add spaces to the pattern for readability -- but in this case, we wanted to match the space between the two words, so we need to escape it with **\\**.

In [34]:

list(finditer(r'(brown | lazy) (dog | fox)', t, X))


[]

In [35]:

list(finditer( r'(brown | lazy)\ (dog | fox)', t, X ))


[<_sre.SRE_Match at 0x7390030>, <_sre.SRE_Match at 0x73900b8>]

Now let's find all the words with an internal **o**.  **\\w+** matches one or more 'word' characters: letters, digits, and underscore.

In [36]:

findall(r'\w+o\w+', t)


['brown', 'fox', 'born', 'dog', 'born']

This one also matches words that begin or end with **o**, because **\\w\*** matches zero or more characters.

In [37]:

findall(r'\w*o\w*', t)


['brown', 'fox', 'born', 'on', 'over', 'dog', 'born', 'on']

### Substitution with **sub**
Now a couple of substitutions... Note that (unlike in Perl) substitution returns a new string.  That's no surprise, because Python strings are immutable.

In [38]:

sub(r'\w*o\w*', 'o-word', t)


'The quick o-word o-word o-word o-word 1/23/2013 jumped o-word the lazy o-word o-word o-word 10/6/10.'

In [39]:

t


'The quick brown fox born on 1/23/2013 jumped over the lazy dog born on 10/6/10.'

### Capturing parts of the pattern match
We can capture things with parentheses, then refer to the captured items by **\1, \2**, etc. in the substitution.

In [41]:

sub(r'(\w*)o(\w*)', r'=\1o\2=', t)


'The quick =brown= =fox= =born= =on= 1/23/2013 jumped =over= the lazy =dog= =born= =on= 10/6/10.'

We can also give things names when we capture them.

In [42]:

sub(r'(?P<before>\w*)o(?P<after>\w*)', r'=\g<before>o\g<after>=', t)


'The quick =brown= =fox= =born= =on= 1/23/2013 jumped =over= the lazy =dog= =born= =on= 10/6/10.'

We can make the pattern, but apparently not the substitution, clearer by adding (?x) and spaces.

In [43]:

sub(r'(?x) (?P<before> \w*) o (?P<after> \w*)', r'=\g<before>o\g<after>=', t)


'The quick =brown= =fox= =born= =on= 1/23/2013 jumped =over= the lazy =dog= =born= =on= 10/6/10.'

Yes, this would have been easier:

In [44]:

sub(r'(?x) (\w*o\w*)', r'=\1=', t)


'The quick =brown= =fox= =born= =on= 1/23/2013 jumped =over= the lazy =dog= =born= =on= 10/6/10.'

### Greedy and lazy matching
Let's try to get the "day" portion of each date. It is the first and only captured group in this pattern.

In [45]:

fi = finditer(r'(?x) / (.* ) /', t)

[m.group(1) for m in fi]


['23/2013 jumped over the lazy dog born on 10/6']

What went wrong? By default, **\*** is greedy.  We can make it lazy by adding **?**.

In [46]:

fi = finditer(r'(?x) / (.*? ) /', t)

[m.group(1) for m in fi]


['23', '6']

This works for the other quantifiers as well.  (**+, ?, {m,n}**)

This one is eager and has an internal **e** in the first match and an internal **o** in the second match.

In [47]:

fi = finditer(r'(?x) ([aeiou]) .* \1', t)

[m.group(0) for m in fi]


['e quick brown fox born on 1/23/2013 jumped over the', 'og born o']

This one is lazy.

In [48]:

fi = finditer(r'(?x) ([aeiou]) .*? \1', t)

[m.group(0) for m in fi]


['e quick brown fox born on 1/23/2013 jumpe', 'over the lazy do', 'orn o']

### split
**split** works differently in **regex**.  Let's start with **re**.
This is a familiar split on whitespace.

In [49]:

split(r'\s+', t)


['The',
 'quick',
 'brown',
 'fox',
 'born',
 'on',
 '1/23/2013',
 'jumped',
 'over',
 'the',
 'lazy',
 'dog',
 'born',
 'on',
 '10/6/10.']

And this splits on the letter **o**:

In [50]:

split(r'o', t)


['The quick br',
 'wn f',
 'x b',
 'rn ',
 'n 1/23/2013 jumped ',
 'ver the lazy d',
 'g b',
 'rn ',
 'n 10/6/10.']

Here's a gotcha:

In [51]:

split(r'o', t, IGNORECASE)


['The quick br',
 'wn f',
 'x born on 1/23/2013 jumped over the lazy dog born on 10/6/10.']

The third argument to **split** does not give options.  It gives the maximum number of splits.  We unknowingly specified 2:

In [52]:

IGNORECASE


2

For split, it is better to use **(?i)** for case-insensitivity.

In [53]:

split(r'(?i)the', t)


['',
 ' quick brown fox born on 1/23/2013 jumped over ',
 ' lazy dog born on 10/6/10.']

Or you can use the **flags** keyword argument.

In [54]:

split(r'the', t, flags=IGNORECASE)


['',
 ' quick brown fox born on 1/23/2013 jumped over ',
 ' lazy dog born on 10/6/10.']

### Capturing in the pattern for **split**

This one splits on words with an internal **u**:

In [55]:

split(r'\w+u\w+', t)


['The ',
 ' brown fox born on 1/23/2013 ',
 ' over the lazy dog born on 10/6/10.']

But what if we wanted to save the **u**-words?  We can use parentheses to capture things inside the splitting pattern!

In [56]:

split(r'(\w+u\w+)', t)


['The ',
 'quick',
 ' brown fox born on 1/23/2013 ',
 'jumped',
 ' over the lazy dog born on 10/6/10.']

In [57]:

split(r'\w*o\w*', t)


['The quick ',
 ' ',
 ' ',
 ' ',
 ' 1/23/2013 jumped ',
 ' the lazy ',
 ' ',
 ' ',
 ' 10/6/10.']

In [58]:

split(r'(\w*o\w*)', t)


['The quick ',
 'brown',
 ' ',
 'fox',
 ' ',
 'born',
 ' ',
 'on',
 ' 1/23/2013 jumped ',
 'over',
 ' the lazy ',
 'dog',
 ' ',
 'born',
 ' ',
 'on',
 ' 10/6/10.']

Here we split on 4-letter words.  Then we do it again and capture the 4-letter words.

In [59]:

split(r'\b\w{4}\b', t)


['The quick brown fox ',
 ' on 1/23/',
 ' jumped ',
 ' the ',
 ' dog ',
 ' on 10/6/10.']

In [60]:

split(r'\b(\w{4})\b', t)


['The quick brown fox ',
 'born',
 ' on 1/23/',
 '2013',
 ' jumped ',
 'over',
 ' the ',
 'lazy',
 ' dog ',
 'born',
 ' on 10/6/10.']

### **split** and zero-width assertions.
Here we split on word boundaries.  **\\b** is a zero-width assertion.  It requires that certain characters be present, but it doesn't "consume" them.

In [61]:

split(r'\b',t)


['The quick brown fox born on 1/23/2013 jumped over the lazy dog born on 10/6/10.']

So what went wrong?  Unlike **split** in Perl, the split function in **re** will not split on zero-width assertions. The new **regex** module gets this right.

In [62]:

from regex import *
split(r'\b',t)


ImportError: No module named regex

Oops. To get the new behavior, we must add the "Version 1" option to the regular expression.  "Version 0" emulates **re**.

In [63]:
split(r'(?V1)\b',t)

error: unexpected end of pattern

We can make life a little easier by setting the version globally.

In [64]:
import regex
regex.DEFAULT_VERSION = VERSION1
split(r'\b',t)

ImportError: No module named regex

**\\m** and **\\M** are zero-width assertions that are true at the beginnings and ends of words.

In [65]:

split(r'\M',t)


['The quick brown fox born on 1/23/2013 jumped over the lazy dog born on 10/6/10.']

In [66]:

split(r'\m',t)


['The quick brown fox born on 1/23/2013 ju',
 'ped over the lazy dog born on 10/6/10.']

### Look-arounds
We can split on any 4-letter word.

In [67]:

split(r'(?x) \b \w{4} \b', t)


['The quick brown fox ',
 ' on 1/23/',
 ' jumped ',
 ' the ',
 ' dog ',
 ' on 10/6/10.']

But what if we want to split on any 4-letter word but **born**? We can use a look-ahead assertion.  Look-aheads and look-behinds come in two flavors: positive and negative.  All four are zero-width assertions.  They required certain characters to be present or absent, but don't consume the characters.  In this case, we need a negative assertion.  We could do a look-ahead:

In [68]:

split(r'(?x) \b (?!born) \w{4} \b', t)


['The quick brown fox born on 1/23/',
 ' jumped ',
 ' the ',
 ' dog born on 10/6/10.']

or a look-behind:

In [69]:

split(r'(?x) \b \w{4} (?<!born) \b', t)


['The quick brown fox born on 1/23/',
 ' jumped ',
 ' the ',
 ' dog born on 10/6/10.']

or, if we are feeling perverse, both:

In [70]:

split(r'(?x) \b (?!bor) \w{4} (?<!orn) \b', t)


['The quick brown fox born on 1/23/',
 ' jumped ',
 ' the ',
 ' dog born on 10/6/10.']

This one splits on any 4-letter word that doesn't contain **o**:

In [71]:

split(r'(?x) \b (?!\w*o) \w{4} \b', t)


['The quick brown fox born on 1/23/',
 ' jumped over the ',
 ' dog born on 10/6/10.']

This one splits on the letter **o**.  The **o** is consumed and lost.

In [72]:

split(r'(?x) o', t)


['The quick br',
 'wn f',
 'x b',
 'rn ',
 'n 1/23/2013 jumped ',
 'ver the lazy d',
 'g b',
 'rn ',
 'n 10/6/10.']

This one has a positive look-ahead assertion.  It splits before every **o**.

In [73]:

split(r'(?x) (?=o)', t)


['The quick brown fox born on 1/23/2013 jumped over the lazy dog born on 10/6/10.']

This one has a positive look-behind assertion.  It splits after evey **o**.

In [74]:

split(r'(?x) (?<=o)', t)


['The quick brown fox born on 1/23/2013 jumped over the lazy dog born on 10/6/10.']

This one splits between  **o** and **r**:

In [75]:

split(r'(?x) (?<=o) (?=r)', t)


['The quick brown fox born on 1/23/2013 jumped over the lazy dog born on 10/6/10.']

The assertions could appear in either order.

In [76]:

split(r'(?x) (?=r) (?<=o)', t)


['The quick brown fox born on 1/23/2013 jumped over the lazy dog born on 10/6/10.']

This one splits between any two consecutive vowels.

In [77]:

split(r'(?x) (?<=[aeiou]) (?=[aeiou])', t)


['The quick brown fox born on 1/23/2013 jumped over the lazy dog born on 10/6/10.']

### Fun with DNA: Open reading frames
DNA is a sequence of bases, A, C, G, or T.  They are translated into proteins 3 bases at a time.  Each 3-base sequence is called a **codon**.  There is a special **start codon** ATG, and three **stop codons**, TGA, TAG, and TAA.  The start and stop codons are highlighted below.

In [80]:

dna = 'cgcgcATGcgcgcgTGAcgcgcgTAGcgcgcgcgc'

dna = dna.lower()


An **opening reading frame** or **ORF** consists of a start codon, followed by some more codons, and ending with a stop codon.  (In real life, "some more" is usually hundreds or thousands.)

In [81]:

orfpat = r'(?x) atg (...)* (tga|tag|taa)'

search(orfpat,dna)


Actually, that's not quite right.  The internal codons should not be stop codons. We can handle that with a negative lookahead assertion.  (Can you think of another way?)

In [82]:

orfpat = r'(?x) atg  ( (?!tga|tag|taa) ... )*  (tga|tag|taa)'

search(orfpat,dna)


We don't really want to capture the "some more codons" separately.  In a minute that will get in the way.  So we can use **(?:...)** to group without capturing:

In [83]:

orfpat = r'(?x) ( atg  (?: (?!tga|tag|taa) ... )*  (?:tga|tag|taa) )'

findall(orfpat,dna)


['atgcgcgcgtga']

Here is another DNA sequence.  Note that this one has overlapping ORFs.  We would like a list of **all** orfs, specifically **ATGcATGcgTGA** and **ATGcgTGAcTAA**.  Our last pattern only finds the first ORF. Since it consumes the first ORF, it also consumes the beginning of the second ORF.

In [84]:

dna = 'cgcgcATGcATGcgTGAcTAAcgTAGcgcgcgcgc'

dna = dna.lower()

findall(orfpat,dna)


['atgcatgcgtga']

Since we want to find something without consuming it, we can use a positive lookahead assertion.  We put the whole ORF pattern inside the lookahead.  We need to capture what is matched by the lookahead without consuming it.

In [85]:

orfpat = r'(?x) (?= ( atg  (?: (?!tga|tag|taa) ... )*  (?:tga|tag|taa) ))'

findall(orfpat,dna)


['atgcatgcgtga', 'atgcgtgactaa']

Notice the position of the capturing parentheses.  This doesn't work:

In [ ]:

orfpat = r'(?x) ( (?= atg  (?: (?!tga|tag|taa) ... )*  (?:tga|tag|taa) ))'

findall(orfpat,dna)


Why? Because the look-ahead assertion has width 0.

### More fun with DNA: Restriction Digest Assays
To perform certain assays, molecular biologists subject DNA sequences to enzymes known as restriction enzymes. There are several types; this is about Type II restriction endonucleases, to be precise. They are usually named with three letters, for the species of origin, and a Roman numeral; e.g., AfeI comes from Alcaligenes faecalis.  These enzymes typically recognize a specific sequence of 6-10 letters, and cut the DNA somewhere in the middle of that sequence.  For example, BgIII recognizes **AGATCT** and cuts between the first **A** and the **G**.

For a typical assay, the DNA will be digested by a "cocktail" of 3-6 enzymes. The lengths of the resulting pieces will be measured by gel electrophoresis.  The lengths should match up with the lengths predicted by an in silico digestion. If not, something is wrong.
Our task is to do the in silico digestion.

For development, here is a dictionary of 4 enzymes, a DNA sequence to digest, and the string we would like to get out of the process. In real life, the DNA sequence would be thousands to ten-thousands of letters long. The researcher could be interested in knowing the cut-points for dozens of enzymes, even though a typical assay uses just a few.

In [86]:

enzymes = {'A-GATCT': 'BgIII',
           'AGC-GCT': 'AfeI',
           'AGG-CCT': 'StuI',
           'AT-CGAT': 'ClaI'}

dna = 'AAAAGCGCTAAAATCGATAAAAAAGATCTAAAAAGCGCT'

goal = 'AAAAGC <AfeI> GCTAAAAT <ClaI> CGATAAAAAA <BgIII> GATCTAAAAAGC <AfeI> GCT'


We are going to use positive look-aheads and look-behinds.  We will build a look ahead-look behind combination for each enzyme.

In [87]:

pats = ['(?<=' + fore + ')(?=' + aft + ')' 
        
        for (fore,aft) in [split(r'-',s) for s in enzymes.keys()]]

pats


['(?<=A)(?=GATCT)', '(?<=AGC)(?=GCT)', '(?<=AT)(?=CGAT)', '(?<=AGG)(?=CCT)']

In [88]:

pats = ' | '.join(pats)
pats


'(?<=A)(?=GATCT) | (?<=AGC)(?=GCT) | (?<=AT)(?=CGAT) | (?<=AGG)(?=CCT)'

In [89]:

pat = "(?x) ( " + pats + " )"
pat


'(?x) ( (?<=A)(?=GATCT) | (?<=AGC)(?=GCT) | (?<=AT)(?=CGAT) | (?<=AGG)(?=CCT) )'

In [90]:
print(goal)

split(pat,dna)


AAAAGC <AfeI> GCTAAAAT <ClaI> CGATAAAAAA <BgIII> GATCTAAAAAGC <AfeI> GCT


['AAAAGCGCTAAAATCGATAAAAAAGATCTAAAAAGCGCT']

It's a good start.  We split in the right places, but we didn't capture the recognition sequences, so we can't retrieve the name of the enzyme from the dictionary. In fact, we captured empty strings.  That's because we captured a zero-width assertion. So we will add some parentheses to capture the look-aheads and look-behinds.

In [91]:

pats = ['(?<=(' + fore + ')) (?=(' + aft + '))' 
        
        for (fore,aft) in [split(r'-',s) for s in enzymes.keys()]]

pats = '  |  '.join(pats)
pat = '(?x) (?: ' + pats + ' )'
pat


'(?x) (?: (?<=(A)) (?=(GATCT))  |  (?<=(AGC)) (?=(GCT))  |  (?<=(AT)) (?=(CGAT))  |  (?<=(AGG)) (?=(CCT)) )'

In [92]:

split(pat,dna)


['AAAAGCGCTAAAATCGATAAAAAAGATCTAAAAAGCGCT']

What happened?  The pattern has eight sets of capturing parentheses. So, the match also returns eight groups when it's executed.  Only the parentheses from the successful alternative will capture anything.  The other six groups are set to **None**.

Happily, **regex** provides a new "branch reset" feature. Briefly, it means that capturing occurs only on the successful branch.

In [93]:

pat = '(?x) (?| ' + pats + ' )'
pat


'(?x) (?| (?<=(A)) (?=(GATCT))  |  (?<=(AGC)) (?=(GCT))  |  (?<=(AT)) (?=(CGAT))  |  (?<=(AGG)) (?=(CCT)) )'

In [94]:

split(pat,dna)


error: unexpected end of pattern

Hooray!  Now all we have to do it to map the recognition sequences into enzyme names:

In [95]:

L = split(pat,dna)

LL = [ ' <' + enzymes[L[i]+'-'+L[i+1]] + '> '+ L[i+2] 
      
                  for i in range(1,len(L),3) ]

L[0] + ''.join(LL)


error: unexpected end of pattern

Then we can pull it all together into a nice class:

In [96]:

import regex as re

class EndonucleaseDigestor:
    
    def __init__(this,enzymeDict):
        pats = ['(?<=(' + fore + '))(?=(' + aft + '))' 
                for (fore,aft) in [re.split(r'-',s) for s in enzymeDict.keys()]]
        pat = ' | '.join(pats)
        pat = '(?x) (?| ' + pat + ' )'
        this.pat = re.compile(pat)
        this.enzymes = enzymeDict
        
    def digest(this,dna):
        L = this.pat.split(dna)
        LL = [ ' <' + enzymes[L[i]+'-'+L[i+1]] + '> '+ L[i+2] for i in range(1,len(L),3) ]
        return L[0] + ''.join(LL)
        

enzymes = {'A-GATCT': 'BgIII',
           'AGC-GCT': 'AfeI',
           'AGG-CCT': 'StuI',
           'AT-CGAT': 'ClaI'}
dna = 'AAAAGCGCTAAAATCGATAAAAAAGATCTAAAAAGCGCT'
goal = 'AAAAGC <AfeI> GCTAAAAT <ClaI> CGATAAAAAA <BgIII> GATCTAAAAAGC <AfeI> GCT'

digestor = EndonucleaseDigestor(enzymes)
if digestor.digest(dna) == goal: print("passed")


ImportError: No module named regex

### TMTOWTDT: **sub** with a function
There's another way to solve the restriction digest problem. This time let's start by building a dictionary that maps the recognition sequences into versions with the enzyme name interposed:

In [97]:

d = { sub('-','',k) : sub('-'," <"+v+"> ", k) for k,v in enzymes.items()}
d


{'AGATCT': 'A <BgIII> GATCT',
 'AGCGCT': 'AGC <AfeI> GCT',
 'AGGCCT': 'AGG <StuI> CCT',
 'ATCGAT': 'AT <ClaI> CGAT'}

And let's build a pattern that matches all the recognition sequences:

In [98]:

p = ' | '.join( [ sub('-','',k) for k in enzymes.keys()])
p = '(?x) (' + p + ')'
p


'(?x) (AGATCT | AGCGCT | ATCGAT | AGGCCT)'

The second argument to **sub** can be a function rather than a string.  If so, the function is called with a **match** object as its argument.  We are interested in the first (and only) thing captured in the match and we want to get the corresponding string out of dictionary **d**.  So we define a function to do that.  Then we call sub with that function.

In [99]:

def subber (m):
    print(m)
    return d[m.group(1)]

sub(p, subber, dna)


'AAAAGC <AfeI> GCTAAAAT <ClaI> CGATAAAAAA <BgIII> GATCTAAAAAGC <AfeI> GCT'

So we can make a class like this:

In [100]:

import regex as re

class AnotherEndonucleaseDigestor:
    
    def __init__(this,enzymeDict):
        this.d = { re.sub('-','',k) : re.sub('-'," <"+v+"> ", k) for k,v in enzymes.items()}
        p = ' | '.join( [ re.sub('-','',k) for k in enzymes.keys()])
        p = '(?x) (' + p + ')'
        this.pat = re.compile(p)
        this.enzymes = enzymeDict
        
    def digest(this,dna):
        return this.pat.sub( lambda m: this.d[m.group(1)]  , dna)

enzymes = {'A-GATCT': 'BgIII',
           'AGC-GCT': 'AfeI',
           'AGG-CCT': 'StuI',
           'AT-CGAT': 'ClaI'}
dna = 'AAAAGCGCTAAAATCGATAAAAAAGATCTAAAAAGCGCT'
goal = 'AAAAGC <AfeI> GCTAAAAT <ClaI> CGATAAAAAA <BgIII> GATCTAAAAAGC <AfeI> GCT'

digestor = AnotherEndonucleaseDigestor(enzymes)
if digestor.digest(dna) == goal: print("passed")


ImportError: No module named regex

That probably seems a lot simpler, but there is one problem.  What if two recognition sites are overlapping?  For the in silico simulation of a real digest, it doesn't much matter, because the resolution of gel electrophoresis is much less than the 5-10 bases that might be overlapping.  On the other hand, if the scientist actually wants a complete inventory of all the restriction sites for a large set of enzymes, overlaps matter, and this solution won't work.

### Nested sets
We have seen some character sets such as **[aeiou]** for all (lower-case) vowels.  Suppose we want all lower-case consonants.  One obvious way is to list them all.  We might also be tempted to use set negation:

In [ ]:

findall(r'[^aeiou]+',t)


The problem is that we get not just consonants, but spaces, digits, etc.
The new **regex** module allows us to do arithmetic on sets:


In [ ]:

findall(r'(?x) [[a-z]--[aeiou]]+', t)


### Fuzzy matching
With **regex**, you can specify that patterns need only be satisfied approximately.  You can specify the number of insertions (**i**), number of deletions (**d**), and number of substitutions (**s**) as well as total number of errors (**e**).
This example allows at most one insertion and at most one deletion for each pattern.

In [ ]:

list(finditer(r'(brown|lazy){i<=1,d<=1} (dog|fox){i<=1,d<=1}',
              
              'The quick crown fax barn on Monday jumped over the sleazy hog bran on Tuesday.'))


You can see that the match object reports the number of insertions, deletions, and substitutions as **fuzzy_counts**. 

You can even **require** a minimum number of errors:

In [ ]:

list(finditer(r'(brown|lazy){1<=e<=3} (dog|fox){1<=e<=2}',
                            
              'The quick crown fax barn on Monday jumped over the sleazy hog bran on Tuesday.'))


What matched what?  We can find out by doing some more capturing.

In [ ]:

findall(r'(?:(brown)|(lazy)){1<=e<=3} (?:(dog)|(fox)){1<=e<=2}',
        
        'The quick crown fax barn on Monday jumped over the sleazy hog bran on Tuesday.')
        

What if we try our orginal correct string? We should get back no matches, because there are no errors, right?  Maybe not.

In [ ]:

findall(r'(?:(brown)|(lazy)){1<=e<=3} (?:(dog)|(fox)){1<=e<=2}',
        
        'The quick brown fox born on 1/23/2013 jumped over the lazy dog born on 10/6/10.')
 

Maybe it will work if we try the **BESTMATCH** option:

In [ ]:

findall(r'(brown|lazy){1<=e<=3} (dog|fox){1<=e<=2}',
        'The quick brown fox born on 1/23/2013 jumped over the lazy dog born on 10/6/10.',
        BESTMATCH)


Hmm.  Maybe we need the **ENHANCEMATCH** option.

In [ ]:

findall(r'(brown|lazy){1<=e<=3} (dog|fox){1<=e<=2}',
        'The quick brown fox born on 1/23/2013 jumped over the lazy dog born on 10/6/10.',
         ENHANCEMATCH)


Maybe we should use both....

In [ ]:

findall(r'(brown|lazy){1<=e<=3} (dog|fox){1<=e<=2}',
        'The quick brown fox born on 1/23/2013 jumped over the lazy dog born on 10/6/10.',
         ENHANCEMATCH | BESTMATCH)


OK, now let's try a spelling corrector.  Here's a list of words.

In [ ]:
f = open('words.txt')

f.readline()


In [ ]:
words = f.readlines()
words[:10]

In [ ]:
words =  ' '.join( [sub('\n', '', w) for w in words] )
words[-60:]

Now let's make a string with some misspelled (and correct) words. It might seem counterintuitive, but we will take the misspelled words and turn them into a pattern, and use the dictionary as the target sequence.

In [ ]:

misspelt = 'abrogatting baandoned abreviat astracted absinthe abussed abus zoan'

misspelt = split('\W+', misspelt)

misspelt = [r"(" + s + r"){e<=2}" for s in misspelt]

misspelt = r"(?x) \m (?: " + " | ".join(misspelt) + r" ) \M"

misspelt


Note that this time we did not use the brach reset feature.  That's because the captured empty strings are going to tell us which misspelled word was matched.

In [ ]:

lis = findall(misspelt,words, ENHANCEMATCH)

lis


Now we will transpose the matrix.  Every column will contain matches for a single misspelled word.  Most of the entries will be empty strings.

In [ ]:

z = list(zip(*lis))

z


And now we can filter out the empty strings.

In [ ]:

z = [ list(filter(lambda s: s!= '', L)) for L in z]

z


It's not entirely satisfactory, but it might work well for correcting the spelling of small sets of words, for example, state names.

### Multiple captures
It's now possible to obtain information on all the successful matches of a repeated capture group, not just the last one.  Use **captures** instead of **group**.

In [ ]:

dna = 'cgcgcATGcgcattcgggcgTGAcgcgcgTAGcgcgcgcgc'
dna = dna.lower()

orfpat = r'(?x) (?= ( atg  ( (?!tga|tag|taa) ... )*  (?:tga|tag|taa) ))'

search(orfpat,dna).captures(2)


In [ ]:

search(orfpat,dna).captures(1)


We can also capture things by name.  The string **s** is an excerpt of a long file describing a gene network.  Each line contains two gene names, and the strength of the connection between them.  In this example, we are only interested in gathering the gene names.

In [ ]:

s = """AT1G01280	AT1G01450	5.1E-3
AT1G01480	AT1G01560	2.3E-2
AT1G01600	AT1G01610	1.6E-2
AT1G01430	AT1G01630	2.1E-2
AT1G01150	AT1G01700	1.1E-2
"""

m = match(
    r'(?x) (?: (?P<geneA>\w+) \s+ (?P<geneB>\w+) \s+ \S+ \n )*',
    s)

m.capturesdict()


We can even reuse a name;

In [ ]:

m = match(
    r'(?x) (?: (?P<gene>\w+) \s+ (?P<gene>\w+) \s+ \S+ \n )*',
    s)

m.capturesdict()


In [ ]:
sorted(set(m.capturesdict()['gene']))

### Reverse searching

Searches can now work backwards:

Note: the result of a reverse search is not necessarily the reverse of a forward search:

In [ ]:

findall(r"(?r)..", "abcde")


Who cares?  I thought of an example.

In [ ]:
sub(r'(?rx) (\d\d\d)',
    r',\1',
    '1 mile = 1760 yards = 5280 ft = 63360 in = 1609344 mm = 160934.4 cm, more or less.  Pi = 3.14159')

In [ ]:
sub(r'(?rx)  (?<=\d) (\d\d\d)',
    r',\1',
    '1 mile = 1760 yards = 5280 ft = 63360 in = 1609344 mm = 160934.4 cm, more or less.  Pi = 3.14159')

In [ ]:
sub(r'(?rx)  (?<! [.] \d*) (?<=\d) (\d\d\d)',
    r',\1',
    '1 mile = 1760 yards = 5280 ft = 63360 in = 1609344 mm = 160934.4 cm, more or less.  Pi = 3.14159')

### POSIX Matching (Leftmost Longest)

The default matching method for alternations is to match the first alternative that will match. The POSIX standard is to find the leftmost longest match. This can be turned on using the POSIX flag **(?p)**.

In [ ]:

list(finditer( r'(dog|doge|doggerel)', 'The doge wrote nothing but doggerel.'))


In [ ]:

list(finditer( r'(?p)(dog|doge|doggerel)', 'The doge wrote nothing but doggerel.'))



### **fullmatch**
The pattern must match the entire string.

In [ ]:

match(r'The doge', 'The doge wrote nothing but doggerel.')


In [ ]:

fullmatch(r'The doge', 'The doge wrote nothing but doggerel.')


Nope, that one didn't match.

### Partial matching
Can the target string be extended to match the pattern?  The optional **partial** argument to **match**, **search**, and **fullmatch** can answer this question. This could be useful if you are validating input from the terminal, for example.

This one is true, because the target can be extended to 'The doge wrote nothing but doggerel.' to match the pattern.  But, if you think about it, you will see that this one is true for any target string.  (It can be extended with 'dogdog'.)

In [ ]:

fullmatch(r'.*dog.*dog.*', 'The doge wrote nothing', partial=True)


This one is more interesting: Can the string be extended to be a Social Security Number?

In [ ]:

fullmatch(r'\d\d\d-\d\d-\d\d\d\d',  "999-89-7", partial=True)


In [ ]:

match(r'\d\d\d-\d\d-\d\d\d\d',  "999-89-7", partial=True)


In [ ]:

fullmatch(r'\d\d\d-\d\d-\d\d\d\d',  "My SSN is 999-89-7", partial=True)


In [ ]:

match(r'\d\d\d-\d\d-\d\d\d\d',  "My SSN is 999-89-7", partial=True)


In [ ]:

search(r'\d\d\d-\d\d-\d\d\d\d',  "My SSN is 999-89-7", partial=True)


Notice that this one is a complete match, so the **partial** field is missing from the match object:

In [ ]:

search(r'\d\d\d-\d\d-\d\d\d\d',  "My SSN is 999-89-7654, but don't tell.", partial=True)


In [ ]:

search(r'\d\d\d-\d\d-\d\d\d\d',  "My SSN is 999-89-76, but don't tell.", partial=True)


### Some functional programming fun

In [ ]:

def twice(f):
    return lambda x: f(f(x))

def prepender(s):
    return lambda t: s + t

twice(twice)(twice(prepender('spam ')))('eggs and spam')


In [ ]:

twice(twice)(twice(prepender(len('spam '))))(len('eggs and spam'))


### Mark's puzzle
Which character is most frequent in a string?

In [ ]:
def most(s, care_about=r'\w'):
    t=''.join(sorted(s))
    p = r'((' + care_about + r')\2*)'
    L = [ m.group(1) for m in finditer(p, t) ]
    m = max(L, key=len)
    return (m[0], len(m))

most('123462232340997092')

In [ ]:
most('123462232340997092', care_about='[13579]')

In [ ]:
most(twice(twice)(twice(prepender('spam ')))('eggs and spam'))

In [ ]:
most(twice(twice)(twice(prepender('spam ')))('eggs and spam'), r'[^aeiou\s]')